In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os

base_dir = "/content/drive/MyDrive/cnn_proje"

folders = [
    "dataset/train",
    "dataset/val",
    "dataset/test",
    "models",
    "notebooks"
]

for folder in folders:
    os.makedirs(os.path.join(base_dir, folder), exist_ok=True)

print("Klasörler hazır ✅")


Klasörler hazır ✅


In [ ]:
import os
from PIL import Image
from collections import Counter

# Senin belirttiğin ana dizin
base_dir = "/content/drive/MyDrive/cnn_proje/dataset/train"

sizes = []

# Tüm alt klasörleri (kategorileri) gez
for root, dirs, files in os.walk(base_dir):
    for filename in files:
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            try:
                file_path = os.path.join(root, filename)
                with Image.open(file_path) as img:
                    sizes.append(img.size) # (genişlik, yükseklik)
            except Exception as e:
                print(f"Hata: {filename} dosyası açılamadı. {e}")

# Sonuçları özetle
if sizes:
    unique_sizes = Counter(sizes)
    print("--- Veri Seti Boyut Analizi ---")
    for size, count in unique_sizes.items():
        print(f"Boyut: {size[0]}x{size[1]} | Adet: {count}")

    if len(unique_sizes) == 1:
        print("\n✅ Harika! Tüm resimler aynı boyutta.")
    else:
        print(f"\n⚠️ Dikkat: {len(unique_sizes)} farklı resim boyutu bulundu. Ön işleme (resize) gerekli.")
else:
    print("❌ Belirtilen klasörde resim bulunamadı. Lütfen yolu kontrol et.")

--- Veri Seti Boyut Analizi ---
Boyut: 512x384 | Adet: 2373

✅ Harika! Tüm resimler aynı boyutta.


In [ ]:
import os, shutil, random

source_dir = "/content/drive/MyDrive/Bitirme dataset/dataset-resized"
target_dir = "/content/drive/MyDrive/cnn_proje/dataset"

train_ratio = 0.7
val_ratio   = 0.15
test_ratio  = 0.15


In [ ]:
source_dir = "/content/drive/MyDrive/Bitirme_dataset/dataset-resized"


In [ ]:
!ls "$source_dir"


In [ ]:
import os
classes = os.listdir(source_dir)
classes


['cardboard', 'glass', 'metal', 'paper', 'plastic']

In [ ]:
import os

target_dir = "/content/drive/MyDrive/cnn_proje/dataset"

for split in ["train", "val", "test"]:
    for cls in classes:
        os.makedirs(os.path.join(target_dir, split, cls), exist_ok=True)

print("Sınıf klasörleri oluşturuldu ✅")


In [ ]:
#trash sınıfı tamamen silindi

!rm -rf /content/drive/MyDrive/cnn_proje/dataset/train/trash
!rm -rf /content/drive/MyDrive/cnn_proje/dataset/val/trash
!rm -rf /content/drive/MyDrive/cnn_proje/dataset/test/trash


In [ ]:
!ls /content/drive/MyDrive/cnn_proje/dataset/train


In [ ]:
import os, shutil, random

source_dir = "/content/drive/MyDrive/Bitirme_dataset/dataset-resized"
target_dir = "/content/drive/MyDrive/cnn_proje/dataset"

train_ratio = 0.7
val_ratio   = 0.15
test_ratio  = 0.15

for cls in classes:
    cls_path = os.path.join(source_dir, cls)
    images = os.listdir(cls_path)
    random.shuffle(images)

    total = len(images)
    train_end = int(total * train_ratio)
    val_end   = train_end + int(total * val_ratio)

    train_imgs = images[:train_end]
    val_imgs   = images[train_end:val_end]
    test_imgs  = images[val_end:]

    for img in train_imgs:
        shutil.copy(os.path.join(cls_path, img),
                    os.path.join(target_dir, "train", cls, img))

    for img in val_imgs:
        shutil.copy(os.path.join(cls_path, img),
                    os.path.join(target_dir, "val", cls, img))

    for img in test_imgs:
        shutil.copy(os.path.join(cls_path, img),
                    os.path.join(target_dir, "test", cls, img))

    print(f"{cls}: {len(train_imgs)} train | {len(val_imgs)} val | {len(test_imgs)} test")


In [ ]:
!ls /content/drive/MyDrive/cnn_proje/dataset/train/cardboard | head


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
base_dir = "/content/drive/MyDrive/cnn_proje/dataset"

IMG_SIZE = (224, 224)
BATCH_SIZE = 32   # 16 yapmaya gerek yok, 32 dengeli

train_datagen = ImageDataGenerator(
    rescale=1./255,

    # Geometrik dönüşümler (gerçek hayata uygun)
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.25,

    # Perspektif & simetri
    shear_range=0.15,
    horizontal_flip=True,

    # Işık koşulları (internetten gelen resimler için ÇOK ÖNEMLİ)
    brightness_range=[0.8, 1.2],

    # Kenar doldurma
    fill_mode="nearest"
)

val_test_datagen = ImageDataGenerator(
    rescale=1./255
)

train_data = train_datagen.flow_from_directory(
    base_dir + "/train",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True
)

val_data = val_test_datagen.flow_from_directory(
    base_dir + "/val",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

test_data = val_test_datagen.flow_from_directory(
    base_dir + "/test",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)


In [ ]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_data.num_classes, activation='softmax')
])


In [ ]:
from PIL import Image
import os

dataset_path = "/content/drive/MyDrive/cnn_proje/dataset"

deleted = 0

for split in ["train", "val", "test"]:
    split_path = os.path.join(dataset_path, split)
    for cls in os.listdir(split_path):
        cls_path = os.path.join(split_path, cls)
        for img_name in os.listdir(cls_path):
            img_path = os.path.join(cls_path, img_name)
            try:
                img = Image.open(img_path)
                img.verify()   # resmi doğrula
            except:
                print("Silindi:", img_path)
                os.remove(img_path)
                deleted += 1

print(f"\nToplam silinen bozuk resim: {deleted}")


In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

val_test_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory(
    "/content/drive/MyDrive/cnn_proje/dataset/train",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True
)

val_data = val_test_gen.flow_from_directory(
    "/content/drive/MyDrive/cnn_proje/dataset/val",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

test_data = val_test_gen.flow_from_directory(
    "/content/drive/MyDrive/cnn_proje/dataset/test",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,              # 5 epoch iyileşme olmazsa dur
    restore_best_weights=True
)

EPOCHS = 50

history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS,
    callbacks=[early_stop]
)


In [ ]:
model.save("/content/drive/MyDrive/cnn_proje/son_veri_isleme.keras")


In [ ]:
test_loss, test_acc = model.evaluate(test_data)
print("Test accuracy:", test_acc)


In [ ]:
#modelin kaydedildiği anlamına gelmektedir
import os

os.path.exists("/content/drive/MyDrive/cnn_proje/son_veri_isleme.keras")


In [ ]:
#confusion matris

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras.models import load_model


In [ ]:
from tensorflow.keras.models import load_model

model = load_model("/content/drive/MyDrive/cnn_proje/son_veri_isleme.keras")


In [ ]:
model.summary()




In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import os

os.listdir("/content/drive/MyDrive/dataset")


In [ ]:
import zipfile

zip_path = "/content/drive/MyDrive/dataset/BitirmeCalismasi.zip"
extract_path = "/content/drive/MyDrive/dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


In [ ]:
import os
os.listdir("/content/drive/MyDrive/dataset")


In [ ]:
"/content/drive/MyDrive/dataset/Garbage classification/Garbage classification"


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(
    "/content/drive/MyDrive/dataset/Garbage classification/Garbage classification",
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
    shuffle=False
)


In [ ]:
from tensorflow.keras.models import load_model

model = load_model("/content/drive/MyDrive/cnn_proje/son_veri_isleme.keras")


In [ ]:
import numpy as np

y_pred = np.argmax(model.predict(test_data), axis=1)
y_true = test_data.classes


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:\n", cm)

print("\nClassification Report:\n")
print(classification_report(
    y_true,
    y_pred,
    target_names=list(test_data.class_indices.keys())
))


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8,6))
sns.heatmap(
    cm,
    annot=True,
    fmt="d",
    cmap="Blues",
    xticklabels=test_data.class_indices.keys(),
    yticklabels=test_data.class_indices.keys()
)
plt.xlabel("Tahmin Edilen Sınıf")
plt.ylabel("Gerçek Sınıf")
plt.title("Confusion Matrix")
plt.show()


In [ ]:
from sklearn.metrics import f1_score

# 1️⃣ Model tahminleri
y_pred = model.predict(test_data)

# 2️⃣ Olasılıklardan sınıf etiketi al
y_pred_classes = y_pred.argmax(axis=1)

# 3️⃣ Gerçek etiketler
y_true = test_data.classes

# 4️⃣ F1-score hesapla
f1_macro = f1_score(y_true, y_pred_classes, average='macro')
f1_weighted = f1_score(y_true, y_pred_classes, average='weighted')

print("Macro F1-score:", f1_macro)
print("Weighted F1-score:", f1_weighted)


In [ ]:
#----------------------------------------------------------------------------------------------------

In [ ]:
#ESP32 CAM BAĞLANTI AŞAMASI

In [ ]:
#-------------------------------------------------------

In [ ]:
model.save("/content/drive/MyDrive/cnn_proje/son_veri_isleme_keras.keras")


In [ ]:
#eğitilmiş modeli tekrar çağırır.
from tensorflow.keras.models import load_model

model = load_model("/content/drive/MyDrive/cnn_proje/son_veri_isleme.keras")


In [ ]:
from google.colab import files

uploaded = files.upload()


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

img_path = list(uploaded.keys())[0]  # yüklenen resim adı

img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = img_array / 255.0  # rescale
img_array = np.expand_dims(img_array, axis=0)


In [ ]:
predictions = model.predict(img_array)
predicted_class_index = np.argmax(predictions)
confidence = np.max(predictions)


In [ ]:
import os

train_dir = "/content/drive/MyDrive/cnn_proje/dataset/train"
class_names = sorted(os.listdir(train_dir))

print(class_names)


In [ ]:
print("Tahmin edilen sınıf:", class_names[predicted_class_index])
print("Güven oranı:", round(confidence * 100, 2), "%")


In [ ]:
#----------------------------------------------------------------------------------------------------

In [ ]:
#2.model eğitimi

In [ ]:
model.save("/content/drive/MyDrive/cnn_proje/cnn_model.keras")


In [ ]:
#modelin kaydedildiği anlamına gelmektedir
import os

os.path.exists("/content/drive/MyDrive/cnn_proje/cnn_model.keras")


In [ ]:
os.listdir("/content/drive/MyDrive/cnn_proje")


In [ ]:
#eğitilmiş modeli tekrar çağırır.
from tensorflow.keras.models import load_model

model = load_model("/content/drive/MyDrive/cnn_proje/cnn_model.keras")


In [ ]:
#test aşaması

In [ ]:
from google.colab import files

uploaded = files.upload()


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

img_path = list(uploaded.keys())[0]  # yüklenen resim adı

img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = img_array / 255.0  # rescale
img_array = np.expand_dims(img_array, axis=0)


In [ ]:
predictions = model.predict(img_array)
predicted_class_index = np.argmax(predictions)
confidence = np.max(predictions)


In [ ]:
import os

train_dir = "/content/drive/MyDrive/cnn_proje/dataset/train"
class_names = sorted(os.listdir(train_dir))

print(class_names)


In [ ]:
print("Tahmin edilen sınıf:", class_names[predicted_class_index])
print("Güven oranı:", round(confidence * 100, 2), "%")
